In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from img_utils import automatic_brightness_and_contrast, find_if_close


In this project, you are to apply image segmentation techniques to X-ray angiography, where X-ray images are taken when an X-ray absorbing substance is injected into the patient's blood stream to produce contrast. The resulting X-ray images have dark regions representing the blood flow within vessels. Your system should be able to automatically locate any occlusion and follow the surrounding vessel wall to compute the ratio between the minimum and nominal vessel diameters. Such results are practically important in detecting coronary disease. Your system should also accept user input of occlusion locations and perform the same percent occlusion measurement in that particular area.

In [ ]:
%matplotlib tk
fig, ax = plt.subplots(4, 6, figsize = (20,10))

for idx, image_name in enumerate(image_names):
    img_dict = {}
    full_path = f'{images_dir}/{image_name}' 
    img = cv2.imread(full_path)
    x_new = int(img.shape[1] * .1)
    y_new = int(img.shape[0] * .1)
    img = img[y_new:img.shape[0] - y_new, x_new: img.shape[1] - x_new]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = automatic_brightness_and_contrast(gray)
    
    blurred = cv2.medianBlur(gray, 5)
    img_dict['blurred'] = blurred
    img_dict['brightness corrected'] = gray
    
    block_size = int(img.shape[0] / 12 )
    block_size = block_size + 1 if block_size % 2 == 0 else block_size
    block_size = block_size + 1 if block_size % 2 == 0 else block_size
    
    edged_adap = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, 10)

    min_contour_area = 1250
    
    thresh = cv2.morphologyEx(edged_adap, cv2.MORPH_ELLIPSE, np.ones((3,3), np.uint8))
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    contours = [i for i in contours if cv2.contourArea(i) > min_contour_area]
    edged_contour = img.copy()
    new_contours = []
    detections = 0
    for idx1, c1 in enumerate(contours):
        cv2.drawContours(edged_contour, [c1], -1, (0,0,0), -1)
        if idx1 == len(contours) - 1:
            break
    edged_contour = cv2.cvtColor(edged_contour, cv2.COLOR_BGR2GRAY)
    threshed = cv2.threshold(edged_contour, 0, 255, cv2.THRESH_BINARY_INV)[1]
    dilated = cv2.dilate(threshed, None, iterations = 2)
    img_dict['edged contour'] = edged_contour
    img_dict['threshed edged contour'] = threshed
    img_dict['dilated'] = dilated
    
    output_image = img.copy()
    thresh = cv2.morphologyEx(dilated, cv2.MORPH_ELLIPSE, np.ones((3,3), np.uint8))
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    contours = [i for i in contours if cv2.contourArea(i) > min_contour_area]
    detections = 0
    for idx1, c1 in enumerate(contours):
        if idx1 == len(contours) - 1:
            break
        for idx2, c2 in enumerate(contours):
            if idx2 == idx1:
                continue
            point = find_if_close(c1, c2, 10)
            if point:
                detections += 1
                center = tuple(contours[idx1][point].squeeze())
                cv2.circle(output_image, center, 25, (0,0,255), 2)
                
    message = f'Possible Disease' if detections > 0 else 'Nothing Detected'
    color = (0,0,255) if detections > 0 else (0,255,0)
    output_image2 = cv2.putText(output_image, message, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    img_dict['output image2'] = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
                
    for idx2, (name, im) in enumerate(img_dict.items()):
        if len(im.shape) == 2:
            ax[idx, idx2].imshow(im, cmap = 'gray')
            ax[idx, idx2].set_title(name)
            continue
        ax[idx, idx2].imshow(im)
        ax[idx, idx2].set_title(name)

# plt.show()

    

    